## SQLite

#### Замеры времени вставки данных в БД SQLite

Справочно: мои значения при вставке 1_000_000 записей в БД на локальном SSD-диске:
- 1.1. sqlite3 execute в цикле: операция вставки заняла 3.7824 секунд
- 2.1. sqlite3 executemany: операция вставки заняла 2.7647 секунд

In [ ]:
import pathlib
import time
import sqlite3


def init_db(filename: str, query: str):
    """Инициализирует базу данных SQLite."""
    if pathlib.Path(filename).exists() and pathlib.Path(filename).is_file():
        pathlib.Path(filename).unlink()

    conn = sqlite3.connect(filename)
    cur = conn.cursor()
    cur.execute(query)
    conn.commit()
    conn.close()


def iter_data(size: int):
    """Генерирует последовательность кортежей с данными пользователя."""
    for i in range(size):
        yield (i, f"user_{i}", f"user_{i}@example.org")


# num_records = 1_000
# num_records = 10_000
num_records = 1_000_000

db_file = "/tmp/test_perfomance.db"

query_create_table = (
    "create table if not exists users(id int, username text, email text)"
)
query_insert_data = "insert into users values (?, ?, ?)"


1.1. Вставка с использованием execute в цикле

In [ ]:
data_to_insert = iter_data(num_records)
init_db(db_file, query_create_table)

conn = sqlite3.connect(db_file)
cur = conn.cursor()
# <measuring time start block>
start_time = time.perf_counter()
for record in data_to_insert:
    cur.execute(query_insert_data, record)
conn.commit()
end_time = time.perf_counter()
# <measuring time end block>
conn.close()
print(
    f"sqlite3 execute в цикле: операция вставки заняла {end_time - start_time:.4f} секунд"
)

2.1. Вставка с использованием executemany

In [ ]:
data_to_insert = iter_data(num_records)
init_db(db_file, query_create_table)

conn = sqlite3.connect(db_file)
cur = conn.cursor()
# <measuring time start block>
start_time = time.perf_counter()
cur.executemany(query_insert_data, data_to_insert)
conn.commit()
end_time = time.perf_counter()
# <measuring time end block>
conn.close()
print(
    f"sqlite3 executemany: операция вставки заняла {end_time - start_time:.4f} секунд"
)